# Predicting Areas of Affluence using Yelp Pricing Data

#### Authors: 
- Eddie Yip
- Hadi Morrow [LinkedIn](https://www.linkedin.com/in/hadi-morrow-4b94164b/) | [GitHub](https://github.com/HadiMorrow) | [Medium](https://medium.com/@hadi.a.morrow)
- Mahdi Shadkam-Farrokhi: [GitHub](https://github.com/Shaddyjr) | [Medium](https://medium.com/@mahdis.pw) | [http://mahdis.pw](http://mahdis.pw)

## Problem Statement [Hadi]

While affluence should never be a factor when choosing to provide disaster aid or not, we must consider the following:

- On the assumption that affluence plays a role, one might relate affluency to preparedness. Those who can afford to will always look out for their families at any cost. Those who can not might not be able to prepare as well due to the fact that it is not an option. 

- On the assumption that affluence is not part of a majority class, if we should be miopic with our search efforts we might want to consider saving the masses, those living in tight coridors and those with little to no income. If effect those most suseptible to losing their lives in a major disaster. 

- Using tax data we aim to show that using YELP data dollar signs is enough to predict where we might want to quickly and accuratly align our efforts. 

New Light Technologies as our audience, we hope to show that while using expensive and hard to handle data such as tax data can be more precise, a quick and dirty aproach could be to simply sord though the dollar signs data on yelp. 


Research cases when affluent areas were better prepared for natural disasters.

We define affluence as any group making over $$100,000 a year (AGI)
If the Yelp price is a significant predictor for this     


## Executive Summary [Mahdi]

- Difficulty gathering data
- Prompt confusing regarding "affluence"
- Other projects used outside data as metric
- We pulled from API and didn't use old data, which was challenging

## Table of Contents
- [Loading Data](#Loading-Data)
- [Preliminary Exploratory Data Analysis](#Preliminary-Exploratory-Data-Analysis)
- [Cleaning the Data](#Cleaning-the-Data)
- [Feature Engineering](#Feature-Engineering)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Model Preparation](#Model-Preparation)
- [Model Selection](#Model-Selection)
- [Model Evaluation](#Model-Evaluation)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)
- [Source Documentation](#Source-Documentation)

## Loading Data
- [All]

In [12]:
import time
import pandas as pd
import numpy as np
import columnExpander
from ast import literal_eval
pd.set_option("display.max_colwidth", 200)

In [67]:
data_file_path = "./data/total_merged.csv"
nyc = pd.read_csv(data_file_path, index_col = 0)

The categories, location, and transactions column is borked and we want them.

In [5]:
df["price"].isnull().sum()

2013

Convert price to binary

Removed non-NY state

Removed area codes that only belong to NYC

Parsing categories

In [6]:
def convert_string_dict_to_string(string, key):
    return ",".join([dic[key] for dic in literal_eval(string)])

df["categories"] = df["categories"].map(lambda s: convert_string_dict_to_string(s,"alias"))

Parsing transactions

### **DataFrame by Eddie from below**

In [ ]:
def convert_string_list_to_string(string):
    return ",".join(literal_eval(string))

df["transactions"] = df["transactions"].map(convert_string_list_to_string)

In [68]:
list_of_zip = [literal_eval(i)['zip_code'] for i in nyc['location']]
list_of_state = [literal_eval(i)['state'] for i in nyc['location']]
list_of_city = [literal_eval(i)['city'] for i in nyc['location']]

In [69]:
nyc['city'] = list_of_city
nyc['state'] = list_of_state
nyc['zip'] = list_of_zip
nyc["area_code"] = nyc["display_phone"].apply(lambda d: d[1:4] if type(d) == str else np.nan)

In [70]:
remove_zip_codes = ['714', '516', '888', '914', '844', '866', '218', '518', '856', '631', '833', '956',
        '219', '607', '201', '781', '732', '908', '862', '973', '254',
        '716', '323', '585', '510', '469', '785', '877', '845', '202',
        '855', '936', '800', '520', '626', '802', '773', '904',
        '203', '312', '353', '717', '302', '374', '484', '708', '954',
        '415', '727', '407', '551', '609', '205', '702', '860']

In [71]:
for i in remove_zip_codes:
    nyc = nyc[nyc['area_code'] != i]

In [72]:
nyc = nyc[nyc['area_code'].notnull()]

In [73]:
nyc['is_closed'].value_counts(dropna = False)

False    10126
True         1
Name: is_closed, dtype: int64

In [74]:
nyc['price'].value_counts(dropna = False)

$$      4781
$       3080
NaN     1576
$$$      562
$$$$     128
Name: price, dtype: int64

In [75]:
nyc.reset_index(drop=True, inplace = True)

In [76]:
nyc.drop([8712], inplace = True)

In [77]:
category = pd.DataFrame([literal_eval(i) for i in nyc['categories']], columns = ['first_category', 'second_category', 'third_category', 'fourth_category'])

In [78]:
category.fillna('None', inplace = True)

In [79]:
category['second_category'] = category['second_category'].apply(lambda x: {'alias': 'None', 'title': 'None'} if x == 'None' else x)
category['third_category'] = category['third_category'].apply(lambda x: {'alias': 'None', 'title': 'None'} if x == 'None' else x)
category['fourth_category'] = category['fourth_category'].apply(lambda x: {'alias': 'None', 'title': 'None'} if x == 'None' else x)

In [80]:
category['first_category_alias'] = [i['alias'] for i in category['first_category']]
category['second_category_alias'] = [i['alias'] for i in category['second_category']]
category['third_category_alias'] = [i['alias'] for i in category['third_category']]

In [81]:
cateories_to_merge = category.drop(columns = ['first_category', 'second_category', 'third_category', 'fourth_category'])

In [82]:
nyc = pd.merge(left = nyc, right = cateories_to_merge, left_index = True, right_index = True)

In [83]:
nyc.drop(columns = ['alias', 'coordinates', 'distance', 'image_url', 'is_closed', 'name','url'], inplace = True)

In [84]:
nyc = pd.get_dummies(nyc, columns = ['first_category_alias', 'second_category_alias','third_category_alias'])

In [92]:
irs = pd.read_csv('./data/NYC IRS.csv')

In [100]:
irs.head(30)

,ZIP code [1],Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents
0,0,Total,"9,468,060","4,848,730","2,939,200","1,480,010","6,084,350","17,229,510","5,371,740"
1,0,"$1 under $25,000",3445310,2354910,445530,595140,2114250,4860280,1503570
2,0,"$25,000 under $50,000",2123960,1161980,441060,474430,1285470,3760760,1210680
3,0,"$50,000 under $75,000",1297130,644420,398720,216490,829250,2370910,677620
4,0,"$75,000 under $100,000",825290,307400,392440,99910,554960,1705470,486790
5,0,"$100,000 under $200,000",1242990,293860,841230,82300,877310,3053530,970230
6,0,"$200,000 or more",533380,86160,420220,11740,423110,1478560,522850
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10001,NaN,"14,520","10,450","2,540","1,120","8,790","19,960","3,290"
9,10001,"$1 under $25,000",3740,2890,350,400,2200,4640,870


In [4]:
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,mos-original-brooklyn,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...","{'latitude': 40.66127, 'longitude': -73.95342}",(718) 513-0698,1542.617156,YwpP-mgXV5N35xhLibLw5g,https://s3-media2.fl.yelpcdn.com/bphoto/-L9roT...,False,"{'address1': '453 Rogers Ave', 'address2': Non...",Mo's Original,1.718513e+10,NaN,4.5,32,[],https://www.yelp.com/biz/mos-original-brooklyn...
1,peaches-hothouse-brooklyn,"[{'alias': 'southern', 'title': 'Southern'}, {...","{'latitude': 40.6833699737169, 'longitude': -7...",(718) 483-9111,3471.525420,GA5msU6NO9rQRctPfDJCBg,https://s3-media1.fl.yelpcdn.com/bphoto/KEAXgZ...,False,"{'address1': '415 Tompkins Ave', 'address2': '...",Peaches HotHouse,1.718484e+10,$$,4.0,1082,"['pickup', 'delivery']",https://www.yelp.com/biz/peaches-hothouse-broo...
2,claw-daddys-brooklyn,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...","{'latitude': 40.68561, 'longitude': -73.98035}",(347) 318-8893,5062.337404,1x2hn3e9sCCZca1cnRTpEg,https://s3-media3.fl.yelpcdn.com/bphoto/ABHo2x...,False,"{'address1': '31 3rd Ave', 'address2': '', 'ad...",Claw Daddy's,1.347319e+10,$$,4.0,282,['restaurant_reservation'],https://www.yelp.com/biz/claw-daddys-brooklyn?...
3,barons-brooklyn,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.6908116, 'longitude': -73.953915}",(718) 230-7100,4451.492133,GxMhN2PEttvw7CRGIzB6Gg,https://s3-media3.fl.yelpcdn.com/bphoto/VmnsId...,False,"{'address1': '564 Dekalb Ave', 'address2': '',...",Baron's,1.718231e+10,$$,4.5,258,"['pickup', 'restaurant_reservation']",https://www.yelp.com/biz/barons-brooklyn?adjus...
4,lowerline-brooklyn-2,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...","{'latitude': 40.67421, 'longitude': -73.96324}",(347) 533-7110,3158.955607,swKXaURwqdSrSTcpHsxdbA,https://s3-media4.fl.yelpcdn.com/bphoto/oJbAhL...,False,"{'address1': '794 Washington Ave', 'address2':...",Lowerline,1.347534e+10,$$,4.5,118,[],https://www.yelp.com/biz/lowerline-brooklyn-2?...


## Preliminary Exploratory Data Analysis
- [All]

## Cleaning the Data
- [Mahdi] one person for consistency

## Feature Engineering
- [All]

## Exploratory Data Analysis
- [Mahdi] killer graphs and visuals

## Model Preparation

## Model Selection
- [Hadi] Exploring models
- [Eddie] Exploring models

Maybe split on which models you 2 want to try out

## Model Evaluation
- [Mahdi] killer graphs and visuals

## Conclusions and Recommendations
- [All]

## Source Documentation